In [1]:
import shutil
import os
import random

original_dir = '/kaggle/input/pix-og/Image_Processing/Original'
pixelated_dir = '/kaggle/input/pix-og/Image_Processing/Pixelated'

test_folder = '/kaggle/working/test'
os.makedirs(test_folder, exist_ok=True)

def copy_images(source_dir, destination_dir, num_images):
    images = os.listdir(source_dir)
    selected_images = random.sample(images, num_images)
    for image_name in selected_images:
        source_path = os.path.join(source_dir, image_name)
        destination_path = os.path.join(destination_dir, image_name)
        shutil.copyfile(source_path, destination_path)

num_images_per_category = 15  

copy_images(original_dir, test_folder, num_images_per_category)

copy_images(pixelated_dir, test_folder, num_images_per_category)

print(f"{num_images_per_category} original and {num_images_per_category} pixelated images copied to the test folder.")

15 original and 15 pixelated images copied to the test folder.


In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import os

model_path = '/kaggle/input/classification-93/pixel_classification_model (2).h5'
model = load_model(model_path)

def preprocess_image(image_path, img_height=224, img_width=224):
    img = image.load_img(image_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  
    img_array = img_array / 255.0  
    return img_array


def classify_image(image_path, model, threshold=0.8):
    img = preprocess_image(image_path)
    prediction = model.predict(img)[0][0]
    if prediction >= threshold:
        return "Pixelated"
    else:
        return "High resolution"

def load_images_from_directory(directory, label):
    image_paths = []
    labels = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
            image_paths.append(os.path.join(directory, filename))
            labels.append(label)
    return image_paths, labels

pixelated_dir = '/kaggle/input/pix-og/Image_Processing/Pixelated'
highres_dir = '/kaggle/input/pix-og/Image_Processing/Original'

pixelated_images, pixelated_labels = load_images_from_directory(pixelated_dir, "Pixelated")
highres_images, highres_labels = load_images_from_directory(highres_dir, "High resolution")

all_images = pixelated_images + highres_images
all_labels = pixelated_labels + highres_labels

label_map = {"Pixelated": 1, "High resolution": 0}
true_labels = [label_map[label] for label in all_labels]


predictions = [classify_image(img_path, model) for img_path in all_images]
predicted_labels = [label_map[pred] for pred in predictions]


2024-07-03 10:20:13.066997: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-03 10:20:13.067152: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-03 10:20:13.256438: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━

In [3]:
print("Classification Report:")
print(classification_report(true_labels, predicted_labels, target_names=["High resolution", "Pixelated"]))

conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(conf_matrix)

report = classification_report(true_labels, predicted_labels, target_names=["High resolution", "Pixelated"], output_dict=True)
precision = report["Pixelated"]["precision"]
recall = report["Pixelated"]["recall"]
f1_score = report["Pixelated"]["f1-score"]
fp = conf_matrix[0][1]

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")
print(f"False Positives: {fp}")

Classification Report:
                 precision    recall  f1-score   support

High resolution       0.94      0.99      0.96      9510
      Pixelated       0.99      0.93      0.96      9510

       accuracy                           0.96     19020
      macro avg       0.96      0.96      0.96     19020
   weighted avg       0.96      0.96      0.96     19020

Confusion Matrix:
[[9440   70]
 [ 627 8883]]
Precision: 0.9921813917122753
Recall: 0.9340694006309148
F1 Score: 0.9622488219682608
False Positives: 70


In [4]:
import time
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import os

model_path = '/kaggle/input/classification-93/pixel_classification_model (2).h5'
model = load_model(model_path)

def preprocess_image(image_path, img_height=224, img_width=224):
    img = image.load_img(image_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  
    img_array = img_array / 255.0  
    return img_array


def classify_image(image_path, model, threshold=0.8):
    img = preprocess_image(image_path)
    start_time = time.time()
    prediction = model.predict(img)
    end_time = time.time()
    fps = 1 / (end_time - start_time)
    
    if prediction >= threshold:
        return "Pixelated", fps
    else:
        return "High resolution", fps

image_path = '/kaggle/input/pixelated/bird.jpg'  
result, fps = classify_image(image_path, model)
print(f"Image classified as {result} with FPS: {fps:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Image classified as Pixelated with FPS: 0.67
